In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from eyeq import Client, Block
from eyeq import block
client = Client("tcp://localhost:13450")

In [ ]:
# Clean-up
def tryit(l):
    try:
        l()
    except:
        pass

tryit(lambda: client.close_stream('raw_stream'))
tryit(lambda: client.close_stream('filtered_stream'))
tryit(lambda: client.close_stream('translate_filter_stream'))
tryit(lambda: client.delete_store("jupyter"))

## Create signal of interest

In [ ]:
def sine(count=block.BLOCK_F32_SAMPLES//2, freq=10.0, offset=0, Fs=200000.0, amplitude=1):
    return amplitude * np.exp(1j*2*np.pi*(np.arange(count) + offset) * freq / Fs, dtype=np.complex64)
def noise(count=block.BLOCK_F32_SAMPLES, scale=0.001):
    return (np.random.default_rng().standard_normal(count, dtype=np.float32) * scale / np.sqrt(2))
def snr2amplitude(snr_db, noise_rms=0.001, signal_rms=1):
    return noise_rms * np.power(10, snr_db / 20) / signal_rms

client.create_store("jupyter", 1000)

offset = 0
cnt = block.BLOCK_I16_SAMPLES // 2
print(cnt)
for i in range(100):
    samples = (noise(count=cnt*2) + sine(count=cnt, freq=-39000, offset=offset, amplitude=np.cos(i * np.pi / 100)).view(np.float32)) * 4000
    bl = block.Block.create(block.BLOCK_TYPE_I16_SAMPLES, data=samples.view(np.float32))
    client.write_block("jupyter", bl)
    offset += block.BLOCK_I16_SAMPLES // 2

In [ ]:
data = client.read_complex_samples("jupyter", 0, 400000)
plt.plot(np.real(data));
plt.plot(np.abs(data));

In [ ]:
plt.plot(np.real(data)[:50]);

## Stream examples

In [ ]:
from scipy.signal import firwin, firwin2

# 127-tap low-pass fir filter with pass-band from -10000 Hz to +10000 Hz.
taps = firwin(127, 0.05)

In [ ]:
plt.plot(taps)

In [ ]:
# Clean-up
try:
    client.close_stream('raw_stream')
    client.close_stream('filtered_stream')
    client.close_stream('translate_filter_stream')
except:
    pass

Create 3 different streams - one just sending through the raw samples, one filtered with a low-pass filter and one frequency shifted and then filtered.

In [ ]:
from eyeq.client import samples

# Raw sample data
raw_stream = client.create_stream(name="raw_stream", layers=[
    samples.StoreReaderStream(name='jupyter', start_block=0, end_block=100)    
])

# Filtered data
filtered_stream = client.create_stream(name="filtered_stream", layers=[
    samples.StoreReaderStream(name='jupyter', start_block=0, end_block=100),
    samples.FirFilterStream(filter_taps=taps.astype(np.complex64).view(np.float32).tolist(), is_complex=True)
])

# Frequency-shifted and filtered data
translate_filter_stream = client.create_stream(name="translate_filter_stream", layers=[
    samples.StoreReaderStream(name='jupyter', start_block=0, end_block=100),
    samples.FrequencyTranslateStream(relative_frequency=39500/200000.0),
    samples.FirFilterStream(filter_taps=taps.astype(np.complex64).view(np.float32).tolist(), is_complex=True)
])

In [ ]:
# Seek to an area of interest

client.seek_stream('raw_stream', 48)
raw_data = client.read_stream('raw_stream', 40000).view(np.complex64)

client.seek_stream('filtered_stream', 48)
fil_data = client.read_stream('filtered_stream', 40000).view(np.complex64)

client.seek_stream('translate_filter_stream', 48)
trans_data = client.read_stream('translate_filter_stream', 40000).view(np.complex64)

The low-pass filtering only allows a small pass-band around 0 Hz through.

In [ ]:
plt.psd(raw_data.view(np.complex64), NFFT=4096);
plt.psd(fil_data.view(np.complex64), NFFT=4096);
#plt.psd(trans_data.view(np.complex64), NFFT=4096);

Frequency translating first and then filtering shifts the signal of interest into the pass band.
As a result, the noise is significantly reduced outside the pass-band, but the signal remains unchanged.

In [ ]:
plt.psd(raw_data.view(np.complex64));
plt.psd(20 * taps.astype(np.complex64) * np.exp(1j * np.pi * 2 * np.arange(len(taps)) * (-39000 / 200000.0)));

In [ ]:
plt.psd(raw_data.view(np.complex64));
plt.psd(trans_data.view(np.complex64));

In [ ]:
plt.plot(np.abs(raw_data));
plt.plot(np.abs(fil_data));
plt.plot(np.abs(trans_data));

In [ ]:
plt.plot(np.real(raw_data));
plt.plot(np.real(trans_data));
plt.plot(np.real(fil_data));